In [ ]:
%pip install --upgrade pip

In [1]:
%pip install tensorflow==2.12 opencv-python mediapipe sklearn matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# § Test if your webcam's feed is accessible
import cv2

cap = cv2.VideoCapture(0)
# ! if this does not open a window with your webcam, try incrementing the '0' to find the right device number
# ? You can replace the '0' with a file path to open a video saved in your local files

while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow("OpenCV feed", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [ ]:
# § Load labels as dict
import json
import os

data_path = r'downloads/data'
labels_file = r'labels.json'

with open(os.path.join(data_path, labels_file), "r") as labels_reader:
    labels = json.load(labels_reader)

labels